# DSCI-100 Project Proposal: Classifying Heart disease

### Introduction:
Explain about heart disease, what variables contribute to heart disease (etc.) Make sure to put in sources



References:

### Data
Column names:
Here are the column names:
- age: age in years
- ca: number of major vessels (0-3)
- chol: serum cholestoral in mg/dl
- cp: chest pain type 
  - Value 1: typical angina
  - Value 2: atypical angina
  - Value 3: non-anginal pain
  - Value 4: asymptomatic
- exang: exercise induced angina 
  - Value 0: no
  - Value 1: yes
- fbs: (fasting blood sugar > 120 mg/dl) 
  - Value 0: false
  - Value 1: true
- num: diagnosis of heart disease 
  - Value 0: < 50% diameter narrowing
  - Value 1: > 50% diameter narrowing
- oldpeak: ST depression induced by exercise relative to rest
- restecg: resting electrocardiographic results
  - Value 0: normal
  - Value 1: having ST-T wave abnormality (T wave inversions and/or ST 
                    elevation or depression of > 0.05 mV)
  - Value 2: showing probable or definite left ventricular hypertrophy
                    by Estes' criteria
- sex: sex
  - Value 0 = female
  - Value 1 = male
- slope: the slope of the peak exercise ST segment
  - Value 1: upsloping
  - Value 2: flat
  - Value 3: downsloping
- thal
  - Value 3: normal
  - Value 6: fixed defect
  - Value 7: reverseable defect
- thalach: Maximum heart rate achieved
- trestbps: resting blood pressure (mm Hg)

Dataset retrieved from: https://archive.ics.uci.edu/ml/datasets/heart+Disease

In [1]:
# Load all libraries
library(repr)
library(tidyverse)
library(tidymodels)
options(repr.matrix.max.rows = 6)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

In [3]:
# Load the dataset and set it's column names accordingly:
heart_all_data <- read_csv("data/processed.switzerland.data",
                  col_names = c("age", "sex", "cp", "trestbps", "chol", 
                              "fbs", "restecg", "thalach", "exang", 
                              "oldpeak", "slope", "ca", "thal", 
                              "num"))
# Change all ? into NA
heart_all_data[heart_all_data == "?"] <- NA

heart_all_data

Rows: 123 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (9): trestbps, fbs, restecg, thalach, exang, oldpeak, slope, ca, thal
dbl (5): age, sex, cp, chol, num

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
32,1,1,95,0,NA,0,127,0,.7,1,NA,NA,1
34,1,4,115,0,NA,NA,154,0,.2,1,NA,NA,1
35,1,4,NA,0,NA,0,130,1,NA,NA,NA,7,3
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
72,1,3,160,0,NA,2,114,0,1.6,2,2,NA,0
73,0,3,160,0,0,1,121,0,0,1,NA,3,1
74,1,2,145,0,NA,1,123,0,1.3,1,NA,NA,1


In [ ]:
# Select and mutate columns that we're interested in into their appropriate datatypes
# Age, oldpeak, cholestrol

heart_data <- heart_all_data |>
              select(age, oldpeak, chol, num) |>
              mutate(a